In [46]:
import torch
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

# Check if CUDA is available and set PyTorch to use GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the pre-trained model (e.g., ResNet)
model = models.resnet50(pretrained=True)

# Move the model to the GPU
model = model.to(device)

# Freeze all the layers in the model
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer of the model to match the number of classes in the new dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # num_classes should be the number of classes in your new dataset

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Load the new training data
train_dir = "images"  # replace train_dir with the path to your data
# train_dir can be a folder with images?
train_data = datasets.ImageFolder(train_dir, transform=transforms.ToTensor())  # replace train_dir with the path to your data

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

# Training the model
num_epochs = 10
train_loss_history = []
for epoch in range(num_epochs):  # num_epochs should be the number of epochs you want to train for
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss_history.append(running_loss/len(train_loader))
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss/len(train_loader)))

# Plotting
plt.figure()
plt.plot(range(1, num_epochs + 1), train_loss_history, label='Train')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()



GPU is not available
Hi
Hi
Hi
Hi
Epoch [1/10], Loss: 0.6967
Hi
Hi
Hi
Hi
Epoch [2/10], Loss: 0.6812
Hi
Hi
Hi
Hi
Epoch [3/10], Loss: 0.6561
Hi
Hi
Hi
Hi
Epoch [4/10], Loss: 0.5862
Hi
Hi
Hi
Hi
Epoch [5/10], Loss: 0.5311
Hi
Hi
Hi
Hi
Epoch [6/10], Loss: 0.5066
Hi
Hi
Hi
Hi
Epoch [7/10], Loss: 0.5086
Hi
Hi
Hi
Hi
Epoch [8/10], Loss: 0.4698
Hi
Hi
Hi
Hi
Epoch [9/10], Loss: 0.4328
Hi
Hi
Hi
Hi
Epoch [10/10], Loss: 0.4792


In [48]:
# Load the testing data
test_dir = "images"  # replace test_dir with the path to your testing data
test_data = datasets.ImageFolder(test_dir, transform=transforms.ToTensor())  # replace test_dir with the path to your testing data

test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

# Switch the model to evaluation mode
model.eval()

# Create a list to store predictions
predictions = []

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Append the predictions to the list
        predictions.extend(predicted.tolist())

# Calculate and print the accuracy
accuracy = correct / total
print('Test Accuracy: %.2f %%' % (100 * accuracy))

# Print the class predictions
print('Class predictions:', predictions)

# Create a reverse mapping from index to class name
idx_to_class = {v: k for k, v in test_data.class_to_idx.items()}

# Transform the predictions list
class_predictions = [idx_to_class[idx] for idx in predictions]

# Print the class predictions
print('Class predictions:', class_predictions)

Test Accuracy: 80.50 %
Class predictions: [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0]
Class predictions: ['wrong_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'wrong_seq', 'correct_seq', 'correct_seq', 'wrong_seq', 'wrong_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'wrong_seq', 'correct_seq', 'correct_seq', 'wrong_seq', 'wrong_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'corre